<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_cams_eac4_retrieve.ipynb"><< 10 - CAMS EAC4 - Retrieve</a>

<div class="alert alert-block alert-warning">
<b>DATA DISCOVERY</b></div>

<br>

## CAMS Global Reanalysis (EAC4) - Stratospheric Ozone - March 2020

This notebook gives an example how `CAMS Global Reanalysis (EAC4)` data can be used to analyse and animate the Arctic ozone hole in March 2020.

The example makes use of matplotlib's function `animation` and Jupyter notebooks function `HTML` to display HTML and video content within a notebook. 

<br>

#### Load required libraries

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes

#### Load helper functions

In [ ]:
%run ./functions.ipynb

<hr>

## Load CAMS Global Reanalysis (EAC4) data for March 2020

The `CAMS Global Reanalysis (EAC4)` data could be not available on the machine. In this case there is the method to download them to [10 - CAMS EAC4 Retrieve](./10_cams_eac4_retrieve.ipynb).

You can load the `CAMS Global Reanalysis (EAC4)` data with xarray's function `xr.open_dataset`. The example data has three dimensions, `latitude`, `longitude` and `time`. You can also see that the EAC data has 62 time steps, with a 12 hourly time step from 01 to 31 March 2020.

In [ ]:
cams_fc = './adaptor.mars.internal-1634575682.2198663-18118-12-b8e3a364-5983-418e-8e08-06361ec5c954.nc'
cams_fc_xr = xr.open_dataset(cams_fc)
cams_fc_xr

<br>

### Load the ozone data variable as `xarray.DataArray`

`gtco3` is the variable name you would like to load as `xarray.DataArray`. You see that the data is disseminated in the unit `kg m**-2`.

In [ ]:
cams_o3 = cams_fc_xr['gtco3']
cams_o3

<br>

### Convert from `kg m**-2` to `Dobson Unit`

Ozone data are often represented in `Dobson Unit`. By dividing the data values with the factor `2.1415*1e-5`, you can easily convert the total column Ozone data from `kg m**-2` to `Dobson Unit`.

In [ ]:
cams_o3_du = cams_o3 / (2.1415*1e-5)
cams_o3_du

<br>

## Visualize the ozone data for one time step with `pcolormesh`

Now, the data is already prepared in a way that you can visualized one time step of the loaded data array. You can use the function [visualize_pcolormesh](./functions.ipynb#visualize_pcolormesh), which makes use of matplotlib's pcolormesh and the Python library `Cartopy` to visualize the data.

With `?visualize_pcolormesh`, you can open the function's docstring in order to see the kwargs required by the function.

In [ ]:
visualize_pcolormesh(data_array = cams_o3_du[50,:,:],
                    longitude = cams_o3_du.longitude,
                    latitude = cams_o3_du.latitude,
                    projection = ccrs.Orthographic(0,90),
                    color_scale = 'jet', 
                    unit = 'DU',
                    long_name = cams_o3.long_name + " " + str(cams_o3_du.time[50].data),
                    vmin = 150, 
                    vmax = 500)

<br>

## Animate the total column ozone over the 31 day reanalysis period

The animation function consists of 4 parts:
- **Setting the initial state:**<br>
 Here, you define the general plot your animation shall use to initialise the animation. You can also define the number of frames (time steps) your animation shall have.
 
 
- **Functions to animate:**<br>
 An animation consists of three functions: `draw()`, `init()` and `animate()`. `draw()` is the function where individual frames are passed on and the figure is returned as image. In this example, the function redraws the plot for each time step. `init()` returns the figure you have defined for the initial state. `animate()` returns the `draw()` function and animates the function over the given number of frames (time steps).
 
 
- **Create a `animate.FuncAnimation` object:** <br>
 The functions defined before are now combined to build an `animate.FuncAnimation` object.
 
 
- **Play the animation as video:**<br>
 As a final step, you can integrate the animation into the notebook with the `HTML` class. You take the generated `animate.FuncAnimation` object and convert it to a HTML5 video with the function `to_html5_video`.

In [ ]:
# Setting the initial state:
# 1. Define figure for initial plot
fig, ax = visualize_pcolormesh(data_array=cams_o3_du[0,:,:],
                               longitude=cams_o3_du.longitude, 
                               latitude=cams_o3_du.latitude,
                               projection=ccrs.Orthographic(0,90), 
                               color_scale='jet', 
                               unit='DU',
                               long_name=cams_o3.long_name + ' '+ str(cams_o3_du.time[0].data), 
                               vmin=150,
                               vmax=500)

frames = 62

def draw(i):
    img = plt.pcolormesh(cams_o3_du.longitude, 
                         cams_o3_du.latitude, 
                         cams_o3_du[i,:,:], 
                         cmap='jet', 
                         transform=ccrs.PlateCarree(),
                         vmin=150,
                         vmax=500)
    
    ax.set_title(cams_o3.long_name + ' '+ str(cams_o3_du.time[i].data), fontsize=20, pad=20.0)
    return img


def init():
    return fig


def animate(i):
    return draw(i)

ani = animation.FuncAnimation(fig, animate, frames, interval=800, blit=False,
                              init_func=init, repeat=True)

HTML(ani.to_html5_video())
plt.close(fig)


<br>

#### Play the animation as HTML5 video

In [ ]:
HTML(ani.to_html5_video())

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_cams_eac4_retrieve.ipynb"><< 10 - CAMS EAC4 - Retrieve</a>

<hr>

<p><img src='./img/all_partners_wekeo.png' align='left' alt='Logo EU Copernicus' width='100%'></img><p>